<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/OpenVLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# T4 GPU

In [ ]:
!pip install -r https://raw.githubusercontent.com/openvla/openvla/main/requirements-min.txt

In [ ]:
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

import torch

# Load Processor & VLA
processor = AutoProcessor.from_pretrained("openvla/openvla-7b", trust_remote_code=True)
vla = AutoModelForVision2Seq.from_pretrained(
    "openvla/openvla-7b",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to("cuda:0")

In [ ]:
image = Image.open("image.jpg")
image

In [ ]:
prompt = "In: What action should the robot take to {<INSTRUCTION>}?\nOut:"

# Predict Action (7-DoF; un-normalize for BridgeData V2)
inputs = processor(prompt, image).to("cuda:0", dtype=torch.bfloat16)
action = vla.predict_action(**inputs, unnorm_key="bridge_orig", do_sample=False)

print(action) # команды

In [ ]:
!pip install pybullet

In [ ]:
import pybullet as p

# Подключение к роботу (пример для Franka Panda)
robot_id = p.loadURDF("franka_panda/panda.urdf")

# Получение управления суставами из предсказанного действия
target_joint_positions = convert_action_to_joints(action)  # Нужна ваша реализация

# Установка целевых положений
for i, pos in enumerate(target_joint_positions):
    p.setJointMotorControl2(robot_id, i, p.POSITION_CONTROL, pos)